# Block End-to-End Machine Learning

## Read File

In [ ]:
import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
import pickle

# Read the data to a pandas data frame
df = pd.read_csv('apartments_data_enriched_lat_lon_combined.csv', sep=',', encoding='utf-8')
# Get number of rows and columns
print(df.shape)
# Zeige die Spaltennamen
print(df.columns)

In [ ]:
# TODO change the file to your own model.
model_filename = "random_forest_regression.pkl"

random_forest_model = RandomForestRegressor()
with open(model_filename, 'rb') as f:
    random_forest_model = pickle.load(f)

print('Number of features: ', random_forest_model.n_features_in_)
print('Features are (see week 1): ', ['rooms', 'area', 'pop', 'pop_dens', 'frg_pct', 'emp', 'tax_income'])
random_forest_model

In [ ]:
df_bfs_data = pd.read_csv('bfs_municipality_and_tax_data.csv', sep=',', encoding='utf-8')
df_bfs_data['tax_income'] = df_bfs_data['tax_income'].str.replace("'", "").astype(float)

In [ ]:
locations = {
    "Zürich": 261,
    "Kloten": 62,
    "Uster": 198,
    "Illnau-Effretikon": 296,
    "Feuerthalen": 27,
    "Pfäffikon": 177,
    "Ottenbach": 11,
    "Dübendorf": 191,
    "Richterswil": 138,
    "Maur": 195,
    "Embrach": 56,
    "Bülach": 53,
    "Winterthur": 230,
    "Oetwil am See": 157,
    "Russikon": 178,
    "Obfelden": 10,
    "Wald (ZH)": 120,
    "Niederweningen": 91,
    "Dällikon": 84,
    "Buchs (ZH)": 83,
    "Rüti (ZH)": 118,
    "Hittnau": 173,
    "Bassersdorf": 52,
    "Glattfelden": 58,
    "Opfikon": 66,
    "Hinwil": 117,
    "Regensberg": 95,
    "Langnau am Albis": 136,
    "Dietikon": 243,
    "Erlenbach (ZH)": 151,
    "Kappel am Albis": 6,
    "Stäfa": 158,
    "Zell (ZH)": 231,
    "Turbenthal": 228,
    "Oberglatt": 92,
    "Winkel": 72,
    "Volketswil": 199,
    "Kilchberg (ZH)": 135,
    "Wetzikon (ZH)": 121,
    "Zumikon": 160,
    "Weisslingen": 180,
    "Elsau": 219,
    "Hettlingen": 221,
    "Rüschlikon": 139,
    "Stallikon": 13,
    "Dielsdorf": 86,
    "Wallisellen": 69,
    "Dietlikon": 54,
    "Meilen": 156,
    "Wangen-Brüttisellen": 200,
    "Flaach": 28,
    "Regensdorf": 96,
    "Niederhasli": 90,
    "Bauma": 297,
    "Aesch (ZH)": 241,
    "Schlieren": 247,
    "Dürnten": 113,
    "Unterengstringen": 249,
    "Gossau (ZH)": 115,
    "Oberengstringen": 245,
    "Schleinikon": 98,
    "Aeugst am Albis": 1,
    "Rheinau": 38,
    "Höri": 60,
    "Rickenbach (ZH)": 225,
    "Rafz": 67,
    "Adliswil": 131,
    "Zollikon": 161,
    "Urdorf": 250,
    "Hombrechtikon": 153,
    "Birmensdorf (ZH)": 242,
    "Fehraltorf": 172,
    "Weiach": 102,
    "Männedorf": 155,
    "Küsnacht (ZH)": 154,
    "Hausen am Albis": 4,
    "Hochfelden": 59,
    "Fällanden": 193,
    "Greifensee": 194,
    "Mönchaltorf": 196,
    "Dägerlen": 214,
    "Thalheim an der Thur": 39,
    "Uetikon am See": 159,
    "Seuzach": 227,
    "Uitikon": 248,
    "Affoltern am Albis": 2,
    "Geroldswil": 244,
    "Niederglatt": 89,
    "Thalwil": 141,
    "Rorbas": 68,
    "Pfungen": 224,
    "Weiningen (ZH)": 251,
    "Bubikon": 112,
    "Neftenbach": 223,
    "Mettmenstetten": 9,
    "Otelfingen": 94,
    "Flurlingen": 29,
    "Stadel": 100,
    "Grüningen": 116,
    "Henggart": 31,
    "Dachsen": 25,
    "Bonstetten": 3,
    "Bachenbülach": 51,
    "Horgen": 295
}

## Random Forest Model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

def model_performance(features, df, random_forest_model = RandomForestRegressor(random_state=42)):
    df = df.sample(frac=1, random_state=42)
    X, y = df[features], df['price']
    scores = cross_val_score(random_forest_model, X, y, scoring="neg_root_mean_squared_error", cv=5)
    print('CV results RMSE:', np.round(scores))
    print('Mean RMSE:', np.mean(np.round(scores, 0)))

In [ ]:
features = ['rooms', 'area', 'pop', 'pop_dens', 'frg_pct', 'emp', 'tax_income']
model_performance(features, df)

## Entfernung zum Stadtzentrum berechnen

In [ ]:
from geopy.distance import geodesic

# Definiere die Koordinaten des Stadtzentrums (Paradeplatz, Zürich)
city_center = (47.3690, 8.5383)

# Funktion zur Berechnung der Entfernung
def calculate_distance(lat, lon):
    return geodesic((lat, lon), city_center).km

# Wende die Funktion auf jede Zeile an, um die Entfernung zu berechnen
df["distance_to_city_center"] = df.apply(lambda row: calculate_distance(row["lat"], row["lon"]), axis=1)

# Zeige die Statistik der neuen Spalte
print(df["distance_to_city_center"].describe())

# Speichere das aktualisierte Dataset
df.to_csv("apartments_data_enriched_lat_lon_combined_with_distance.csv", index=False)


## Modell erneut trainieren mit der neuen Variable

### Spalte ins Feature-Set aufnehmen

In [ ]:
# Features definieren
features = ['rooms', 'area', 'distance_to_city_center']

# X = Features, y = Zielvariable (Preis)
X = df[features]
y = df["price"]

### Train/Test-Split

In [ ]:
from sklearn.model_selection import train_test_split

# Split in Trainings- und Testdaten (80/20)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

### Modell trainieren (Random Forest)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Modell definieren
model = RandomForestRegressor(n_estimators=100, random_state=42)

# Modell trainieren
model.fit(X_train, y_train)

# Vorhersagen machen
y_pred = model.predict(X_test)

# RMSE berechnen
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"RMSE mit distance_to_city_center: {rmse}")

In [ ]:
import pickle

# Speichere das Modell in einer Pickle-Datei
model_filename = "random_forest_regression_distance_to_city_center.pkl"
with open(model_filename, 'wb') as f:
    pickle.dump(model, f)

print(f"Modell wurde in {model_filename} gespeichert.")

random_forest_model_distance_to_city_center = RandomForestRegressor()
with open(model_filename, 'rb') as f:
    random_forest_model_distance_to_city_center = pickle.load(f)

print('Number of features: ', random_forest_model_distance_to_city_center.n_features_in_)
print('Features are: ', ['rooms', 'area','distance_to_city_center'])
random_forest_model_distance_to_city_center

In [ ]:
import gradio as gr

# Funktion für die Preisvorhersage
def predict_price(rooms, area, distance_to_city_center):
    # Modellvorhersage
    input_data = np.array([[rooms, area, distance_to_city_center]])
    prediction = model.predict(input_data)
    return f"Geschätzter Mietpreis: {round(prediction[0], 2)} CHF"

# Gradio-Interface
app = gr.Interface(
    fn=predict_price,
    inputs=[
        gr.Number(label="Anzahl Zimmer"),
        gr.Number(label="Fläche in m²"),
        gr.Number(label="Entfernung zum Stadtzentrum (km)")
    ],
    outputs=gr.Textbox(label="Vorhersage"),
)

app.launch()